In [1]:
from pathlib import Path

import typer
import pandas as pd
import verifiers as vf
from dotenv import load_dotenv
from openai import AsyncOpenAI


assert load_dotenv(), "Failed to load .env file"

In [2]:
import mlflow

# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Enable autologging with all features
mlflow.openai.autolog()
# Create a unique name for your experiment.
mlflow.set_experiment("rlvr-eval")

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1756636926792, experiment_id='2', last_update_time=1756636926792, lifecycle_stage='active', name='rlvr-eval', tags={}>

In [3]:
openai_client = AsyncOpenAI()

models_response = await openai_client.models.list()
available_models = [item.id for item in models_response.data if item.object == "model"]
print("Available models:", available_models)

model = available_models[0]
print("Using model:", model)


Available models: ['Qwen/Qwen2.5-7B-Instruct']
Using model: Qwen/Qwen2.5-7B-Instruct


In [4]:
datasets_str = "bdsaglam/musique-mini,answerable,validation"
noise_rate = 1.0
retriever = "hybrid"
temperature = 0.5
max_new_tokens = 1024

print("🔮 Starting MuSiQue evaluation")
print("=" * 50)
print(f"📝 Model: {model}")
print(f"📊 Dataset: {datasets_str} (noise rate: {noise_rate})")
print(f"🔍 Retriever: {retriever}")
print(f"🌡️ Temperature: {temperature}")
print(f"🎯 Max tokens: {max_new_tokens}")
print("=" * 50)

# Load MuSiQue environment
print("🌍 Loading MuSiQue environment...")

vf_env = vf.load_environment(
    env_id="vf-musique-structured",
    datasets_str=datasets_str,
    noise_rate=noise_rate,
    retriever_name=retriever,
)
print(f"✅ Environment loaded with {len(vf_env.dataset)} examples")

# Use OpenAI-compatible API client (e.g., for vLLM)
print("🤖 Using OpenAI-compatible API client...")

# Run evaluation using the environment
print("🔄 Running evaluation...")
results = vf_env.evaluate(
    openai_client,
    model,
    rollouts_per_example=1,
    sampling_args={"temperature": temperature, "max_tokens": max_new_tokens},
)


🔮 Starting MuSiQue evaluation
📝 Model: Qwen/Qwen2.5-7B-Instruct
📊 Dataset: bdsaglam/musique-mini,answerable,validation (noise rate: 1.0)
🔍 Retriever: hybrid
🌡️ Temperature: 0.5
🎯 Max tokens: 1024
🌍 Loading MuSiQue environment...


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Filter:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

2025-09-11 11:11:22 - verifiers.envs.MuSiQueEnv - INFO - eval_dataset is not set, falling back to train dataset


✅ Environment loaded with 100 examples
🤖 Using OpenAI-compatible API client...
🔄 Running evaluation...


Evaluating 100 rollouts: 100%|██████████| 100/100 [00:00<00:00, 1041.50it/s]


[Trace(trace_id=tr-bc83fa1d25911bf6f2f814ee6fc29adc), Trace(trace_id=tr-620344eabbac060d6d894df7aeb77236), Trace(trace_id=tr-fb3dd743e17f6ddff69aaf802313b040), Trace(trace_id=tr-667ef204664b7fa35fa5fe00caab7d4c), Trace(trace_id=tr-798bac8e343d18bf709cd752944b7c4e), Trace(trace_id=tr-b41244c248ff1dec677c5ef95b3406ff), Trace(trace_id=tr-06b34e48983edb6efb3451f66d1d6651), Trace(trace_id=tr-351ee64a0c4f65ea79e485e65dac8f16), Trace(trace_id=tr-004a774926cd42ec9bad19b560df7501), Trace(trace_id=tr-d44c88505c32586b3914f4991823d1fc)]

In [5]:
results.metrics.keys()

dict_keys(['exact_match_reward', 'f1_reward', 'retrieval_recall_reward', 'retrieval_precision_reward', 'citation_reward', 'format_reward', 'combined_reward'])

In [6]:
ds = vf_env.make_dataset(results)
ds

Dataset({
    features: ['prompt', 'completion', 'answer', 'task', 'reward', 'info', 'exact_match_reward', 'f1_reward', 'retrieval_recall_reward', 'retrieval_precision_reward', 'citation_reward', 'format_reward', 'combined_reward'],
    num_rows: 100
})

In [7]:
ds.to_json("../outputs/musique-eval-results.jsonl", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

2833139

In [8]:
df = ds.to_pandas()
df

,prompt,completion,answer,task,reward,info,exact_match_reward,f1_reward,retrieval_recall_reward,retrieval_precision_reward,citation_reward,format_reward,combined_reward
0,[{'content': 'Answer the question based on the...,"[{'content': '', 'role': 'assistant', 'tool_ca...",LeMat revolver,default,0.000000,"{'answers': ['lemat revolver', 'LeMat revolver...",0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
1,[{'content': 'Answer the question based on the...,"[{'content': '', 'role': 'assistant', 'tool_ca...",Vitor Constâncio,default,0.175000,"{'answers': ['vitor constâncio', 'Vitor Constâ...",0.0,0.000000,0.5,0.500000,0.0,0.0,0.175000
2,[{'content': 'Answer the question based on the...,"[{'content': '', 'role': 'assistant', 'tool_ca...",45,default,0.641667,"{'answers': ['45'], 'docs': [{'body': 'Weber C...",0.0,0.666667,1.0,0.666667,1.0,1.0,0.641667
3,[{'content': 'Answer the question based on the...,"[{'content': '', 'role': 'assistant', 'tool_ca...",three different relationships he had in the past,default,0.350000,{'answers': ['three different relationships he...,0.0,0.000000,1.0,1.000000,0.0,0.0,0.350000
4,[{'content': 'Answer the question based on the...,"[{'content': '', 'role': 'assistant', 'tool_ca...",Michael Bublé,default,0.000000,"{'answers': ['Michael Bublé', 'michael bublé']...",0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,[{'content': 'Answer the question based on the...,"[{'content': '', 'role': 'assistant', 'tool_ca...",two alliances,default,0.200000,"{'answers': ['two alliances'], 'docs': [{'body...",0.0,0.000000,0.5,0.500000,0.0,1.0,0.200000
96,[{'content': 'Answer the question based on the...,"[{'content': '', 'role': 'assistant', 'tool_ca...",John Kukuzelis,default,0.175000,"{'answers': ['john kukuzelis', 'John Kukuzelis...",0.0,0.000000,0.5,0.500000,0.0,0.0,0.175000
97,[{'content': 'Answer the question based on the...,"[{'content': '', 'role': 'assistant', 'tool_ca...",Vito Corleone,default,0.175000,"{'answers': ['vito andolini', 'Vito Andolini C...",0.0,0.000000,0.5,0.500000,0.0,0.0,0.175000
98,[{'content': 'Answer the question based on the...,"[{'content': '', 'role': 'assistant', 'tool_ca...",The Île de la Cité,default,0.175000,"{'answers': ['île de la Cité', 'Île de la Cité...",0.0,0.000000,0.5,0.500000,0.0,0.0,0.175000


In [9]:
df.describe()

,reward,exact_match_reward,f1_reward,retrieval_recall_reward,retrieval_precision_reward,citation_reward,format_reward,combined_reward
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.323571,0.110000,0.159425,0.660000,0.607000,0.175000,0.330000,0.323571
std,0.231518,0.314466,0.338031,0.300505,0.282298,0.296231,0.472582,0.231518
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.175000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.175000
50%,0.275000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.275000
75%,0.350000,0.000000,0.000000,1.000000,1.000000,0.500000,1.000000,0.350000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
row = df.iloc[0]